In [1]:
from policyengine_us import Simulation
import plotly.graph_objects as go
import pandas as pd
from policyengine_core.charts import format_fig 

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_situation(has_child=False):
    """Create a situation dictionary for either single adult or adult with child"""
    members = ["you"]
    if has_child:
        members.append("your first dependent")
    
    situation = {
        "people": {
            "you": {
                "age": {"2025": 40}
            }
        },
        "families": {
            "your family": {"members": members}
        },
        "spm_units": {
            "your household": {"members": members}
        },
        "tax_units": {
            "your tax unit": {"members": members}
        },
        "households": {
            "your household": {
                "members": members,
                "state_name": {"2025": "CA"}
            }
        },
        "marital_units": {
            "your marital unit": {"members": ["you"]}
        },
        "axes": [[{
            "name": "employment_income",
            "count": 620,
            "min": 0,
            "max": 310000
        }]]
    }
    
    if has_child:
        situation["people"]["your first dependent"] = {"age": {"2025": 10}}
    
    return situation


In [3]:
situation_single_adult = create_situation(has_child=False)
situation_one_child = create_situation(has_child=True)

In [4]:
simulation_one_child = Simulation(
    situation=situation_one_child,
)

simulation_single_adult = Simulation(
    situation=situation_single_adult,
)


In [5]:
VARIABLES =[
    "household_net_income",
    "employment_income",
    "ctc_value",
]

one_child_df = simulation_one_child.calculate_dataframe(VARIABLES, map_to = "household", period = 2025)
single_adult_df = simulation_single_adult.calculate_dataframe(VARIABLES, map_to = "household", period = 2025)


In [6]:
# Add children indicator
single_adult_df['children'] = 0
one_child_df['children'] = 1

# Combine dataframes
combined_df = pd.concat([single_adult_df, one_child_df])
combined_df = combined_df.reset_index()

In [7]:
BLUE_PRIMARY = "#2C6496"
DARK_GRAY = "#616161"

In [8]:
# Create Plotly figures
# Figure 1: Household Post-Tax Income vs. Gross Income
fig1 = go.Figure()

# Add traces for each child scenario
for children_count in [0, 1]:
    df_subset = combined_df[combined_df['children'] == children_count]
    
    label = f"{children_count} child" if children_count == 1 else f"{children_count} children"
    
    fig1.add_trace(go.Scatter(
        x=df_subset['employment_income'],
        y=df_subset['household_net_income'],
        mode='lines',
        name=label,
        line=dict(color=BLUE_PRIMARY if children_count == 1 else DARK_GRAY),
        hovertemplate='<b>%{fullData.name}</b><br>' +
                    'Household gross income: %{x:$,.0f}<br>' +
                    'Household net income: %{y:$,.0f}' +
                    '<extra></extra>'
    ))
# Add 45-degree line
fig1.add_trace(go.Scatter(
    x=combined_df['employment_income'],
    y=combined_df['employment_income'],
    mode='lines',
    name='45° line',
    line=dict(dash='dash', color='black'),
    hoverinfo='skip'
))

fig1.update_layout(
    title='Household Post-Tax Income vs. Gross Income (by Children)<br>Single Filer in CA (filing as Head of Household if has Children)',
    xaxis_title='Household Gross Income',
    yaxis_title='Household Net Income',
    showlegend=True,
    template='plotly_white',
    hovermode='closest'
)

fig1 = format_fig(fig1)
fig1.show()

In [9]:
# Figure 2: Difference in Post-Tax Income
# Create figure
fig2 = go.Figure()

# Calculate differences using the original dataframes
total_difference = one_child_df['household_net_income'] - single_adult_df['household_net_income']
ctc_difference = one_child_df['ctc_value'] - single_adult_df['ctc_value']
difference_without_ctc = total_difference - ctc_difference

# Use employment_income from either dataframe as the x-axis (they're the same)
x_values = one_child_df['employment_income']

# Add trace for total difference (with all benefits)
fig2.add_trace(go.Scatter(
    x=x_values,
    y=total_difference,
    mode='lines',
    name='Actual (including Child Tax Credit, Head of Household, etc.)',
    line=dict(color=BLUE_PRIMARY),
    hovertemplate='<b>%{fullData.name}</b><br>' +
                  'Household gross income: %{x:$,.0f}<br>' +
                  'Difference in net income: %{y:$,.0f}' +
                  '<extra></extra>'
))

# Add trace for difference without CTC
fig2.add_trace(go.Scatter(
    x=x_values,
    y=difference_without_ctc,
    mode='lines',
    name='Without Child Tax Credit',
    line=dict(color=DARK_GRAY),
    hovertemplate='<b>%{fullData.name}</b><br>' +
                  'Household gross income: %{x:$,.0f}<br>' +
                  'Difference in net income: %{y:$,.0f}' +
                  '<extra></extra>'
))

fig2.update_layout(
    title='Difference in Post-Tax Income between 1 Child and 0 Children<br>Single Filer in CA (filing as Head of Household if has Children)',
    xaxis_title='Household Gross Income',
    yaxis_title='Difference in Household Net Income<br>(1 child - 0 children)',
    showlegend=True,
    template='plotly_white',
    hovermode='closest',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig2.update_layout(
    title='Difference in Post-Tax Income between 1 Child and 0 Children<br>Single Filer in CA (filing as Head of Household if has Children)',
    xaxis_title='Household Gross Income',
    yaxis_title='Difference in Household Net Income<br>(1 child - 0 children)',
    showlegend=True,
    template='plotly_white',
    hovermode='closest'
)

fig2 = format_fig(fig2)
fig2.show()
